1.Suppose you have a daily univariate time series, and you want to forecast the next seven days. Which RNN architecture should you use, i.e. how many neurons you plan to set up?

A)LSTM - Long short term memory networks is applied in time series forecasting. This Model solve various problem of univariate time series forecasting .  This model is divided into a various section like Data preparation,Vanilla LSTM,stacked LSTM,Bidirectional LSTM,CNN LSTM,ConvLSTM.Before univariate time series forcasting Data preparation is applied because LSTM maps a past observation as an input to output observation.Vanillia LSTM is a layer which is used to make a predication.Stacked LSTM means that various multiple layer are stack on the top of another layer.Bidirectional LSTM is used to some sequence prediction to backwork and forward both direction.CNN LSTM convolutional neural network that is used in two dimensional image data.ConvLSTM is used to convolutional reading of input.

It is dependent upon dataset and layers though if it is Vanilla LSTM, then I can go for 50 neurons. But
in nature it would be variable can't say exact number for any network.
    


2.Why do people use Encoder–Decoder RNNs rather than plain sequence-to-sequence RNNs for automatic translation?


A)It is better to use Encoder-Decoder RNN rather than plain seq2seq becuase Encoder-decoder RNN can process(read-translate) whole sentence at time, while seq2seq RNN can only process one word at a time.

Below are some of the reasons why people prefer using Encoder-Decoder RNNs rather than plain sequence to sequence RNNs:

1.It has the ability of training a single end-to-end model directly on the sentences of destination and the start.    
2.It can efficiently handle input and output texts of variable lengths with ease.   
3.During encryption, the text of variable lengths is encoded in a way that it can be stored in a fixed-length vector.    
4.And during the decryption, data stored in the fixed-length vector are extracted and decrypted into the text of variable lengths.    
5.It is also trained to predict the next character in the sequence using the previous characters of the sequence.

3.(optional) Install the tensorflow_addons (pip install tensorflow-addons) and test the Python script of automatic translation (RNN_demo5.ipynb). 

In [9]:
import sys
assert sys.version_info >= (3, 5)
# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    IS_COLAB = True
except Exception:
    IS_COLAB = False
# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"
if not tf.test.is_gpu_available():
    
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        
           print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
# Common imports
import numpy as np
import os
# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    
    
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


In [10]:
tf.random.set_seed(42)


In [11]:
vocab_size = 100
embed_size = 10

In [12]:
import tensorflow_addons as tfa

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler,
                                                 output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, initial_state=encoder_state,
    sequence_length=sequence_lengths)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.models.Model(
    inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
    outputs=[Y_proba])

In [13]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [14]:
X = np.random.randint(100, size=10*1000).reshape(1000, 10)
Y = np.random.randint(100, size=15*1000).reshape(1000, 15)
X_decoder = np.c_[np.zeros((1000, 1)), Y[:, :-1]]
seq_lengths = np.full([1000], 15)
history = model.fit([X, X_decoder, seq_lengths], Y, epochs=2)

Train on 1000 samples
Epoch 1/2
1000/1000 [==============================] - 17s 17ms/sample - loss: 4.6050
Epoch 2/2
1000/1000 [==============================] - 13s 13ms/sample - loss: 4.6038
